In [20]:
# h1ttps://eprint.iacr.org/2015/525

# 4.1
from py_ecc.bn128 import G1, G2, pairing, add, multiply, neg, curve_order, eq, final_exponentiate, Z1, is_inf

# functions
def string_to_int(message):
    return int.from_bytes(message, byteorder='big')

# keygen
def user_setup():
    x = 10
    y = 15

    X2 = multiply(G2, x)
    Y2 = multiply(G2, y)

    sk1 = {"x": x, "y": y}
    pk2 = {"X2": X2, "Y2": Y2}

    return sk1, pk2

def sign(sk1, message):
    h = 77
    H1 = multiply(G1, h)
    s2 = multiply(H1, sk1['x'] + sk1['y'] * string_to_int(message))
    return {"s1": H1, "s2": s2}

def verify(pk2, message, sigma):
    assert not eq(sigma['s1'],G1)
    lhs2 = add(pk2['X2'], multiply(pk2['Y2'], string_to_int(message)))
    lhs_pairing = pairing(lhs2, sigma['s1'])
    rhs_pairing = pairing(G2, sigma['s2'])
    assert(eq(lhs_pairing, rhs_pairing)), "pairing verification failed"


def execute():
    # Your main code here
    print("This is the main function.")

    sk1, pk2 = user_setup()
    m1 = b"sam polgar"
    sigma = sign(sk1, m1)
    m2 = b"not sam polgar" # change this to m1 to verify
    verify(pk2, m2, sigma)

execute()

This is the main function.


AssertionError: pairing verification failed

In [23]:
# # https://eprint.iacr.org/2015/525

# # 4.2
# from py_ecc.bn128 import G1, G2, pairing, add, multiply, neg, curve_order, eq, final_exponentiate
# from py_ecc.bn128 import G1, G2, pairing, add, multiply, neg, curve_order, eq
# import hashlib

# # functions


# def string_to_int(message):
#     return int.from_bytes(message, byteorder='big')

# # keygen


# # random field elements to scale G2
# x = 10
# y1 = 15
# y2 = 20
# y3 = 25
# y4 = 30

# X = multiply(G2, x)
# Y1 = multiply(G2, y1)
# Y2 = multiply(G2, y2)
# Y3 = multiply(G2, y3)
# Y4 = multiply(G2, y4)

# sk = [x, y1, y2, y3, y4]
# pk = [X, Y1, Y2, Y3, Y4]

# # sign
# m1 = b"sam"
# m2 = b"polgar"
# m3 = b"student"
# m4 = b"1/1/2000"

# m1_int = string_to_int(m1)
# m2_int = string_to_int(m2)
# m3_int = string_to_int(m3)
# m4_int = string_to_int(m4)

# h = 77
# H = multiply(G1, h)

# h_scalar = x + y1 * m1_int + y2 * m2_int + y3 * m3_int + y4 * m4_int
# H = multiply(G1, h_scalar)

# # verify






# h1ttps://eprint.iacr.org/2015/525

# 4.2

# helper functions
def string_to_int(message):
    return int.from_bytes(message, byteorder='big')

def mod_scalar(a):
    return int(a % curve_order)

def ecmul(point, scalar):
    if(is_inf(point) or scalar == 0):
        return Z1
    return multiply(point, mod_scalar(int(scalar)))

def ecadd(s, *args):
    if(s == Z1) and not args:
        return Z1

    for x in args:
        if x == Z1:
            continue  # skip adding Z1
        if isinstance(x, tuple) and len(x) == 2:
            # Directly add if x is a point on the curve
            s = add(s, x)
        else:
            # Assume x is a scalar and try multiplying with G1
            try:
                s = add(s, ecmul(G1, x))
            except Exception:
                # If it fails, try with G2
                s = add(s, ecmul(G2, x))
    return s

# keygen
def user_setup():
    x = 10
    y1 = 15
    y2 = 20
    y3 = 25

    X2 = multiply(G2, x)
    Y1 = multiply(G2, y1)
    Y2 = multiply(G2, y2)
    Y3 = multiply(G2, y3)

    sk1 = {"x": x, "y1": y1, "y2": y2, "y3": y3}
    pk2 = {"X2": X2, "Y1": Y1, "Y2": Y2, "Y3": Y3}

    return sk1, pk2


def sign(sk1, m1, m2, m3):
    h = 77
    H1 = multiply(G1, h)
    s2 = multiply(H1, sk1['x'] + sk1['y1'] * string_to_int(m1)
                  + sk1['y2'] * string_to_int(m2)
                  + sk1['y3'] * string_to_int(m3))
    return {"s1": H1, "s2": s2}


def verify(pk2, m1, m2, m3, sigma):
    assert not eq(sigma['s1'], G1)
    lhs2 = ecadd(pk2['X2'],
                 ecmul(pk2['Y1'], string_to_int(m1)),
                 ecmul(pk2['Y2'], string_to_int(m2)),
                 ecmul(pk2['Y3'], string_to_int(m3))
                 )
    lhs_pairing = pairing(lhs2, sigma['s1'])
    rhs_pairing = pairing(G2, sigma['s2'])
    assert(eq(lhs_pairing, rhs_pairing)), "pairing verification failed"


def execute():
    # Your main code here
    print("This is the main function.")

    sk1, pk2 = user_setup()
    m1 = b"sam polgar"
    m2 = b"student"
    m3 = b"1/1/2000"
    sigma = sign(sk1, m1, m2, m3)
    m_fake = b"not sam polgar"  # change this to m1 to verify
    verify(pk2, m1, m2, m3, sigma)


execute()

This is the main function.
